In [1]:

import pandas as pd
import numpy as np
import os
import sys
import itertools
from idd_forecast_mbp import constants as rfc
from idd_forecast_mbp.parquet_functions import read_parquet_with_integer_ids, write_parquet
import glob
cause_map = rfc.cause_map
cause = "malaria"
reference_age_group_id = cause_map[cause]['reference_age_group_id']
reference_sex_id = cause_map[cause]['reference_sex_id']
PROCESSED_DATA_PATH = rfc.PROCESSED_DATA_PATH
as_full_cause_df_path = f'{PROCESSED_DATA_PATH}/as_full_{cause}_df.parquet'
full_2023_hierarchy_path = f"{PROCESSED_DATA_PATH}/full_hierarchy_2023_lsae_1209.parquet"
hierarchy_df = read_parquet_with_integer_ids(full_2023_hierarchy_path)
forecast_years = list(range(2022, 2101))
md_gbd_location_df = hierarchy_df[hierarchy_df['most_detailed_gbd'] == True].copy()
last_year = min(forecast_years)
md_gbd_location_filter = ('location_id', 'in', md_gbd_location_df['location_id'].unique().tolist())
last_year_filter = ('year_id', '==', last_year)
as_md_gbd_malaria_df = read_parquet_with_integer_ids(as_full_cause_df_path,
                                                   filters = [md_gbd_location_filter, last_year_filter])
base_md_gbd_malaria_df = as_md_gbd_malaria_df[(as_md_gbd_malaria_df['age_group_id'] == reference_age_group_id) &
                                              (as_md_gbd_malaria_df['sex_id'] == reference_sex_id)].copy()
base_md_gbd_malaria_df = base_md_gbd_malaria_df.rename(columns={
    'malaria_inc_rate': 'base_malaria_inc_rate',
    'malaria_mort_rate': 'base_malaria_mort_rate'}).reset_index(drop=True)

as_md_gbd_malaria_df = as_md_gbd_malaria_df.merge(base_md_gbd_malaria_df[['location_id', 'year_id', 'base_malaria_inc_rate', 'base_malaria_mort_rate']],
                                                   on=['location_id', 'year_id'], how='left')
as_md_gbd_malaria_df['rr_inc_as'] = as_md_gbd_malaria_df['malaria_inc_rate'] / as_md_gbd_malaria_df['base_malaria_inc_rate']
as_md_gbd_malaria_df['rr_mort_as'] = as_md_gbd_malaria_df['malaria_mort_rate'] / as_md_gbd_malaria_df['base_malaria_mort_rate']



In [2]:
as_md_gbd_malaria_df_path = f"{PROCESSED_DATA_PATH}/as_md_gbd_malaria_df.parquet"

In [3]:
write_parquet(as_md_gbd_malaria_df, as_md_gbd_malaria_df_path)

✅ Metadata validation passed for /mnt/team/idd/pub/forecast-mbp/02-processed_data/as_md_gbd_malaria_df.parquet


True